<a href="https://colab.research.google.com/github/dding-g/Cat_Dog_Classification/blob/master/Cat_Dog_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dog, cat image classification problem set


### [CUDA](http://pytorch.org/docs/stable/cuda.html)

* cuda를 이용하겠습니다


In [0]:
import torch
import numpy as np

# check if CUDA is available
train_on_gpu = torch.cuda.is_available()
#train_on_gpu = False

if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')

---
## Load the [Data](http://pytorch.org/docs/stable/torchvision/datasets.html)

* 아미지를 다운로드 받습니다
* 폴더별로
 - test
 - train
 - validation

 data를 받습니다. 


In [0]:
#!pip install googledrivedownloader

In [0]:
from os.path import exists
from google_drive_downloader import GoogleDriveDownloader as gdd
import tarfile 

#if exists("./Cat_Dog_data.tgz"):
#    !rm -rf ./Cat_Dog_data.tgz

gdd.download_file_from_google_drive(file_id='1WpY0qpe7yF5C5M52z1BMIzYVpDYiU3OV',
                                   dest_path = './Cat_Dog_data.tgz')

tf = tarfile.open("Cat_Dog_data.tgz")
tf.extractall()

## Problem 1 [20 points]: 

* Training, validation, test를 위한 dataloader, transform을 적절하게 준비해주세요
* 아래 data 준비하는 코딩을 수행하고, 아래 markdown에 준비한 과정 및 이유를 구체적으로 설명하세요
* 아래 답안 작성에 data의 구조에 대해서 설명하세요
* 코드에는 주석을 달아주세요.

In [0]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt
import torch
from torchvision import datasets
import torchvision.transforms as transforms
import helper


# 데이터들을 load 할때 적용시키는 transform. 일정한 규격을 만들어줌
transform_train_validation = transforms.Compose([transforms.RandomRotation(30),
                                                 transforms.RandomResizedCrop(128),
                                                 transforms.RandomHorizontalFlip(),
                                                 transforms.ToTensor(),
                                                 transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                                                                      std=[0.229, 0.224, 0.225])
                                                # 정규화
                                                # RGB 픽셀들 - mean / std 인데, 학습시킬때 중구난방한 featre들을 응집시켜주는 역할을 한다.
                                                # 해당 정규화 값은 RGB image 를 normalize 시키는데 널리 사용되는 값이며
                                                # pytorch document에 명시되어 있다.
                                                 ])

#Test데이터를 위한 transform. Test데이터 이기 때문에 크기만 맞춰주고
#다른 변화는 주지 않는다.
transform_test = transforms.Compose([
                                     transforms.ToTensor(),
                                     transforms.RandomResizedCrop(128),
                                     transforms.RandomHorizontalFlip(),
                                     transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
                                     ])

batch_size = 10
base_dir = '/content' # Colab의 base Directory

# Train 데이터 load
dataset_trainig = datasets.ImageFolder(base_dir + '/train', transform=transform_train_validation)
dataloader_training = torch.utils.data.DataLoader(dataset_trainig, batch_size=batch_size, shuffle=True)

# Test 데이터 load
dataset_test = datasets.ImageFolder(base_dir + '/test', transform=transform_train_validation)
dataloader_test = torch.utils.data.DataLoader(dataset_test, batch_size=batch_size)

# Validation 데이터 load
dataset_validation = datasets.ImageFolder(base_dir + '/validation', transform=transform_train_validation)
dataloader_validation = torch.utils.data.DataLoader(dataset_validation, batch_size=batch_size, shuffle=True)

# Dog, Cat의 class 지정. Test 할때 labeling으로 사용
classes = ['dog', 'cat']

**분석 및 설명:**
===
* Train, Validation 은 같은 `Transform`으로 불러온다. 

* 이때 처음에는 244 x 244, 255 x 255 등 크기를 상대적으로 크게 `resize`해서 불러왔으나, colab의 cuda 메모리 오류 때문에 크기를 128까지 줄였다. dataset이 
적지 않으므로 학습에 크게 영향이 없을것이라 생각했다. 
  * 나중에 안 사실이지만 위의 정규화 값과 함께 널리 쓰이는 `resize` 형식이 244 x 244 라고 한다. [pytorch DOC](https://pytorch.org/docs/stable/torchvision/models.html) 에 나와있다. 여기서는 memory 제한 때문에 크기를 조금 작게 수정했다.

* 많은 데이터를 학습시키려면 일반 `GradientDescent`로는 시간이 오래걸리기 때문에 `SGD`를 사용한다. `batch size`를 1000를 주어보았지만 성능이 너무 않좋아져서 20까지 내렸다. 

* 처음에는 전부 0.5로 정규화 값을 주고 `normalization`를 시켰으나, batch size 조절, layer 수정, epoch 수정 등 많은 노력에도 일정 수치 이하로 떨어지지 않아 image 정규화를 다시 시키는 작업을 했다.
  * [pytorch DOC](https://pytorch.org/docs/stable/torchvision/models.html) 에 정규화 값이 나와있는데 이는 pytorch에서 이미지를 정규화 시키는데 널리 사용된다고 한다. 




## Problem 2 [20 point]: Define the Network Architecture

* 구현하고자하는 network을 작성해주세요
* 아래 구현 방법과 이유를 구체적으로 설명하세요
* 코드에는 주석을 달아주세요. 
* 아래 모델을 구체적으로 설명하고, 설정 이유를 작성해주세요
* 본 과정에서는 직접 network을 구현하고, transfer learning은 활용하지 않도록 하겠습니다.

In [0]:
#@title 기본 제목 텍스트
# 코드 작성
import torch.nn as nn
import torch.nn.functional as F

# define the CNN architecture
class Net(nn.Module):
   # 레이어 정의
    def __init__(self):  
        super(Net, self).__init__()
        # input shape : 128 x 128 x 3
        # conv1 filter : 7 x 7, input chanel size : 3
        # Conv2d(input chanal size, filter chanel size, filter size, padding)
        self.conv1 = nn.Conv2d(3, 32, 3, padding = 1)# 결과 size : 128 x 128 x 16
        self.conv2 = nn.Conv2d(32, 64, 3, padding = 1) # 결과 size : 128 x 128 x 32
        self.conv3 = nn.Conv2d(64, 128, 3, padding = 1) # 결과 size : 128 x 128 x 64

        # conv1 -> pool -> conv2 -> pool -> conv3 -> pool
        #pool 때문에 해상도를 절반으로 줄임. 
        #128 X 128    64 x 64    32 x 32    16 x 16(최종 size)
        
        self.pool = nn.MaxPool2d(2,2) # 해상도를 절반으로 줄이는 작업. 연산량 줄이기 위함

        self.fc1 = nn.Linear(256 * 128, 256 * 32)
        self.fc2 = nn.Linear(256 * 32, 2)

        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))

        x = x.view(-1, 256 * 128) #vector를 펴는 작업
        x = self.dropout(x)

        x = F.relu(self.fc1(x))
        x = self.dropout(x)

        x = F.log_softmax(self.fc2(x), dim = 1)
        return x 

# create a complete CNN
model = Net()
print(model)

# move tensors to GPU if CUDA is available
if train_on_gpu:
    model.cuda()  


Net(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=32768, out_features=8192, bias=True)
  (fc2): Linear(in_features=8192, out_features=2, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)


**분석 및 설명:**
===
* Conv layer를 3 곂으로 쌓고 커널 사이즈는 3x3, stride = 1, padding = 1 그리고 채널 수는 두배씩 늘려갔다. 
* filter size는 3과 7중 어느것이 더 성는이 좋은가를 보기위해 실험했는데 크게 성능차이가 나지는 않았다. 
* size를 계산하기 쉽도록 padding 을 조절하고 높은 해상도로는 연산량이 많아지므로 pool로 크기를 2배씩 줄인다. 
* overfitting을 방지하기위해 dropout을 주었으며 full connected 후에 softmax 함수를 씌워 확률값을 알 수 있도록 하였다.



## Problem 3 [5 point]: Specify Loss Function

* Loss 함수와 optimizer를 구현하세요
* 선정 이유를 설명하세요


In [0]:
# 코드

import torch.optim as optim

# specify loss function (categorical cross-entropy)
criterion = nn.NLLLoss()

# specify optimizer
optimizer = optim.SGD(model.parameters(), lr=0.008)

 **설명:**
 ===

 * `SGD`는 `batch size`를 1로 주는 `GD`와 달리 `batch size`를 크게 주어 탐색 속도를 늘린다. 렌덤하게 데이터 값을 받아오고 해당 데이터를 가지고 최적의 해(weight)를 찾기 때문에 `batch size`보다는 정확도가 낮을 수 있지만, 빠르게 학습할 수 있다는 장점이 있다. 따라서 20,000개의 데이터를 학습시키기에 `batch size`를 1로 주는건 너무 가혹하다고 생각해 `SGD`를 채택하였다.
 
 * 우리는 Dog, Cat 2가지 Class 중 하나를 예측하는 일을 원한다. 따라서 Class 분류에 특화되어 있는 `Cross-entropy` loss function을 사용하겠다. 내가 학습한 결과와 input으로 들어오는 값을 곱해 loss를 계산하고 이 과정에서 `Gradient`를 구해 최적 `weight`를 구하는데 사용한다.


## Problem 4 [30 point]: Train the Network

* training loss와 validation loss를 기록하며 training을 구현하세요
* 만약 validation loss가 최소화된 모델을 저장하세요
* 코드에는 모두 주석을 포함해주세요
* training과정을 설명하고, training 결과를 분석해주세요

In [0]:
# 코드 작성
 
# 반복 횟수 지정
n_epochs = 60

valid_loss_min = np.Inf # valid_loss값의 최소 값을 지정

# epoch 만큼 반복
for epoch in range(1, n_epochs+1):

    # train_loss와 valid_loss를 미리 init
    train_loss = 0.0
    valid_loss = 0.0
    
    ###################
    # train the model #
    ###################
    model.train()
    # batch size 만큼 data를 load하기 때문에 batch size만큼 반복
    for data, target in dataloader_training: 
        # move tensors to GPU if CUDA is available
        if train_on_gpu:
            data, target = data.cuda(), target.cuda()

        # 학습했던 gradient를 초기화하고 새로 학습할 준비를 함
        optimizer.zero_grad()
        # 현재 불러온 image를 기반으로 forward를 호출함. CNN 학습
        output = model(data)
        # 학습이 끝난 후 계산 결과와 현재 불러온 image의 label(0 or 1)을 이용해 loss를 계산.
        loss = criterion(output, target)
        # loss 값들을 이용해 Gradient 계산
        loss.backward()
        # hyper parameter들을 update한다.
        optimizer.step()
        # training loss 값을 update 한다.
        train_loss += loss.item()*data.size(0)
        
    ######################    
    # validate the model #
    ######################
    model.eval()
    validation_iter = iter(dataloader_validation)
    for data, target in validation_iter:
        # move tensors to GPU if CUDA is available
        if train_on_gpu:
            data, target = data.cuda(), target.cuda()
        # 현재 불러온 image를 기반으로 forward를 호출함. CNN 학습
        output = model(data)
        # 학습이 끝난 후 계산 결과와 현재 불러온 image의 label(0 or 1)을 이용해 loss를 계산.
        loss = criterion(output, target)
        # validation loss 값을 update 한다.
        valid_loss += loss.item()*data.size(0)
    
    # Trainloss 와 Validation loss의 평균을 구함
    train_loss = train_loss/len(dataloader_training.sampler)
    valid_loss = valid_loss/len(dataloader_validation.sampler)
        
    # print training/validation statistics 
    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
        epoch, train_loss, valid_loss))
    
    # save model if validation loss has decreased
    if valid_loss <= valid_loss_min:
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
        valid_loss_min,
        valid_loss))
        torch.save(model.state_dict(), 'model_cifar3_ir0001.pt')
        valid_loss_min = valid_loss

Epoch: 1 	Training Loss: 0.664063 	Validation Loss: 0.639547
Validation loss decreased (inf --> 0.639547).  Saving model ...
Epoch: 2 	Training Loss: 0.628244 	Validation Loss: 0.601568
Validation loss decreased (0.639547 --> 0.601568).  Saving model ...
Epoch: 3 	Training Loss: 0.601969 	Validation Loss: 0.583861
Validation loss decreased (0.601568 --> 0.583861).  Saving model ...
Epoch: 4 	Training Loss: 0.584682 	Validation Loss: 0.552799
Validation loss decreased (0.583861 --> 0.552799).  Saving model ...
Epoch: 5 	Training Loss: 0.569176 	Validation Loss: 0.560006
Epoch: 6 	Training Loss: 0.556928 	Validation Loss: 0.571247
Epoch: 7 	Training Loss: 0.546492 	Validation Loss: 0.542372
Validation loss decreased (0.552799 --> 0.542372).  Saving model ...
Epoch: 8 	Training Loss: 0.538259 	Validation Loss: 0.527349
Validation loss decreased (0.542372 --> 0.527349).  Saving model ...
Epoch: 9 	Training Loss: 0.527392 	Validation Loss: 0.504275
Validation loss decreased (0.527349 --> 0.

**분석 및 설명:**
==

* ephoc 만큼 반복하며 학습한다. 
  - 1회 학습마다 loss를 batch size 만큼 데이터를 불러와 학습시킨다. 
  - 이때 loss는 학습이 끝난 `output` 데이터와 현재 이미지로 불러와 있는 `target(label)`을 이용해 `Cross entropy` 방식으로 loss를 갱신한다.
* validation도 마찬가지로 batch size만킄 데이터를 불러오고, 불러온 데이터들을 학습이 끝난 `output`데이터들과 `validation` 데이터들을 이용해 loss를 갱신한다. 이렇게 되면 해당 image에 대해 학습한 결과, 테스트한 결과를 동시에 알 수 있으므로 테스트의 진행방향이 어떤지 사용자가 추측할수있다.




## Problem 5 [5 point] Validation Loss가 최소화된 Model가져오기

* 최소 validation loss를 갖는 model을 불러옵니다

In [0]:
# 코드 작성
model.load_state_dict(torch.load('model_cifar3_ir0001.pt'))

<All keys matched successfully>

---
## Problem 6 [20point]: Test the Trained Network

* Test set을 활용하여 성능 검증
* Accuracy (분류 성공률)와 test loss를 출력하세요
* 코드에는 주석을 달아주세요
* 아래 test 결과에 대해서 간단하게 설명/분석 해주세요

In [0]:
# 코드 작성
# track test loss
test_loss = 0.0
class_correct = list(0. for i in range(2))
class_total = list(0. for i in range(2))

model.eval()
# iterate over test data
for data, target in dataloader_test:
    # move tensors to GPU if CUDA is available
    if train_on_gpu:
        data, target = data.cuda(), target.cuda()
    # forward pass: compute predicted outputs by passing inputs to the model
    output = model(data)
    # calculate the batch loss
    loss = criterion(output, target)
    # update test loss 
    test_loss += loss.item()*data.size(0)
    # convert output probabilities to predicted class
    _, pred = torch.max(output, 1)    
    # compare predictions to true label
    correct_tensor = pred.eq(target.data.view_as(pred))
    correct = np.squeeze(correct_tensor.numpy()) if not train_on_gpu else np.squeeze(correct_tensor.cpu().numpy())
    # calculate test accuracy for each object class
    for i in range(batch_size):
        label = target.data[i]
        class_correct[label] += correct[i].item()
        class_total[label] += 1

# average test loss
test_loss = test_loss/len(dataloader_test.dataset)
print('Test Loss: {:.6f}\n'.format(test_loss))

for i in range(2):
    if class_total[i] > 0:
        print('Test Accuracy of %5s: %2d%% (%2d/%2d)' % (
            classes[i], 100 * class_correct[i] / class_total[i],
            np.sum(class_correct[i]), np.sum(class_total[i])))
    else:
        print('Test Accuracy of %5s: N/A (no training examples)' % (classes[i]))

print('\nTest Accuracy (Overall): %2d%% (%2d/%2d)' % (
    100. * np.sum(class_correct) / np.sum(class_total),
    np.sum(class_correct), np.sum(class_total)))

Test Loss: 0.287689

Test Accuracy of   dog: 90% (1128/1250)
Test Accuracy of   cat: 84% (1057/1250)

Test Accuracy (Overall): 87% (2185/2500)


**분석 및 설명:**
==
* Test데이터를 가지고 위의 학습을 통해 weight를 잘 찾았는지 알아본다. 
 - 위에서 했던 방식과 똑같이 데이터를 불러오고, 학습한 데이터와 test tartget(label)을 이용해 `Cross entropy`방식으로 loss를 갱신한다.
 - 데이터들이 얼마나 잘 맞았는지 알아보는 Accuracy를 계산하기 위해 학습한 `outout`과 입력받은 `target`의 label을 비교해, 예측을 성공한 케이스의 수와 전체 수를 구해 저장 및 연산한다.

---
## Problem 7: 전체적인 총평

* Data준비, Training과 validation 과정에서 성능 개선을 위해서 수행한 과정과 성공/실패 이유를 분석해주세요


**설명:**
==
1. 데이터 준비
  * 훈련의 다양화를 위해 `location, RandomResize, RandomHorizonCrop`들을 적용시켰다.
  * 연산의 부담을 덜기위해 이미지를 resize 할때 128 x 128 로 비교적 작게 주었다. 사진을 실제로 뽑아보니 10개중 7개 정도 비율로 얼굴이 잘 나와있는걸 볼 수 있었다.
  * batch size는 처음에 200, 500, 1000 등 상당히 크게 주었다. `SGD`의 특성상 큰 `Batch size`를 주어도 된다는 생각이었는데 이는 우리의 모델이 잘 구성되었을때 통용하는 이야기 였다. loss값이 점점 커지는걸 보고 최대한 모델을 잘 만들어 보고 `Batch size`는 조금 작게 주었다.
  ---
2. 학습
* epoch를 60회 정도로 주고 학습시켜서 기록을 보고 괜찮은 epoch이 어느정도인지 파악한다.
* 처음에는 ConvLayer를 5 ~ 6 곂으로 쌓아 채널 수를 늘려보고, `full connected`할때 사용되는 `nn.Linear`의 parameter들을 조정해보기도 했다.  `Cuda OutofMemory Error`와 별로 개선되지 않는 loss를 보고 다른 방법을 찾았다.
* 가장 효과적인 방법은 `Normalization`을 제대로 해 주는 것 이다. 위에서도 말했듯이 RGB image를 `Normalization`할때 널리 사용되는 `mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]` 값으로 바꾸어 주었다. 원래는 0.5 로 `Normalization` 해주었는데, 생각해보니 이는 흑백 사진. 즉 값의 분포에 대한 변수가 없는 이진 데이터에 대해서만 효과가 있다.
* 다음은 `batch size`를 조절했다. 기존의 100회 이상의 size에서 30,20,10 으로 점차 내렸다. `BGD`와 `SGD`의 특성을 생각해 정확도를 높이기 위해 size를 줄였다.
* `batch size`가 20인 경우
 - test accuracy가 dog : 88%, cat : 80%, total : 84% / training loss : 0.253, validation loss : 0.254 가 나왔다. 
* `batch size`가 10인 경우
 - test accuracy가 dog : 90%, cat : 86%, total : 88% 가 나왔다.
* learning rate을 0.01로 두고 60회 epoch를 돌려 loss를 0.25 까지 떨구었다.
 + 더 떨구기 위해 조금 더 촘촘한 작업이 필요하다고 생각했다. 그래서 SGD의 Iearning rate 을 0.01 -> 0.001로 낮춰 gradient를 더 촘촘하게 찾을 수 있도록 하였다. 하지만 
loss가 떨어지는 속도는 너무 느렸고 충분한 성능을 낼 수 없었다. 2만개의 데이터를 Iearning rate  0.001로 학슴시키기엔 충분한 epoch를 낼 수 없어서(학습 속도가 너무 느려서 많이 줄 수 없었다.) Ir을 0.008 정도로 주고 학습을 다시 시켜 보았다.
* Iearning rate을 0.008로 준 경우
  - test accuracy가 dog : 87%, cat : 89%, total : 88% 가 나왔다.
  - epoch를 더 주고 학습을 시키면 더 좋은 성능을 낼 것 같았지만 이정도로만 학습시켜도 5시간정도 걸려서 더 늘리는건 힘들것같다.
  - 성능 차이를 보기위해 7x7 사이즈로 주었던 filter size를 3x3으로 바꾸었다. 결과는 dog : 90%, cat : 84%, total : 87% 로 큰 차이는 없는것으로 보였다.

  ---
3. 결과

* 결과는 강아지 분류 90%, 고양이 분류 84%로 그닥 좋지 않은 성능을 보였다. 정확도를 더 올리기 위해서는 epoch을 좀더 늘리고 model을 Conv Layer를 추가하거나, image resize를 좀 크게 해주거나 하는 방식으로 최대한 효과적으로 구성해야 하는데, 메모리 등 제한적인 환경에서 진행하다 보니 Layer의 크기, image의 크기 등 더 많은 시도를 하지 못해 아쉽다.